# Footprint analysis

The footprints are derived from the APPLAUSE table created by this query:

```
SELECT DISTINCT p.telescope, p.ota_aperture, p.date_orig, e.ut_mid,
    e.exptime, p.plate_format, pl.emulsion, p.plate_id, sc.filename_scan,
    s.ra_icrs, s.dec_icrs, s.fov1, s.fov2,
    s.pixel_scale, sc.naxis1, sc.naxis2, s.stc_polygon, 
    p.observatory, p.site_name, p.site_longitude, p.site_latitude
FROM applause_dr4.plate AS p
LEFT JOIN applause_dr4.exposure AS e USING (plate_id)
LEFT JOIN applause_dr4.solution AS s USING (plate_id)
LEFT JOIN applause_dr4.scan AS sc USING (plate_id)
LEFT JOIN  applause_dr4.plate AS pl USING (plate_id)
WHERE p.ota_aperture > 0.6
AND s.header_wcs IS NOT NULL
AND e.exptime > 300
AND p.numexp < 2
AND e.year_mid < 1958.0
ORDER BY p.ota_aperture, p.telescope, p.date_orig, p.plate_format;
```

The purpose is to find which plates share overlapping footprints.

In [1]:
import os, io
import warnings
import csv

import numpy as np

import pandas as pd

from astropy import units as u
from astropy.io import fits
from astropy.io.fits import Header, Card
from astropy.table import Table, unique, vstack
from astropy.time import Time
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Longitude, Latitude, EarthLocation
from astropy.utils.exceptions import AstropyWarning

from regions import PolygonSkyRegion
from mocpy import MOC

from erfa import ErfaWarning
from earthshadow import get_shadow_center, get_shadow_radius, dist_from_shadow_center

from settings import DATAPATH, fname, get_table_psf_nomatch

# Total sky area in steradians
ALL_SKY = 4 * np.pi * u.steradian

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.simplefilter('ignore', category=ErfaWarning)
warnings.simplefilter('ignore', category=FutureWarning)

## Functions

In [3]:
def compute_moc(table, row):
    e = table['stc_polygon'][row].split()

    ra  = Longitude([e[2], e[4], e[6], e[8]], unit=u.deg)
    dec = Latitude ([e[3], e[5], e[7], e[9]], unit=u.deg)
    
    moc = MOC.from_polygon(ra, dec)
    
    area = moc.sky_fraction * ALL_SKY
    area_deg2 = area.to(u.deg**2).value

    return moc, area_deg2

In [4]:
def compute_moc_intersection(moc1, moc2):

    intersection_moc = moc1.intersection(moc2)

    area = intersection_moc.sky_fraction * ALL_SKY
    area_deg2 = area.to(u.deg**2).value
    
    return intersection_moc, area_deg2

In [5]:
def compute_earth_shadow(table, row_index):
    
#     longitude = 10.242    # TODO: take this from main csv input file instead
#     latitude  = 53.482

    longitude = table['site_longitude'][row_index]
    latitude  = table['site_latitude'][row_index]

    ra  = table['ra_icrs'][row_index]
    dec = table['dec_icrs'][row_index]

    location = EarthLocation.from_geodetic(longitude, latitude, 0.0)
    es_radius = get_shadow_radius(orbit='GEO', geocentric_angle=False)

    time_date = Time(table['ut_mid'][row_index])
    
    dist = dist_from_shadow_center(ra, dec, time=time_date, obs=location, orbit='GEO')

    return dist

In [6]:
def scan_table(table, result, match_area):

    # pre-compute a footprint moc and an Earth's shadow angular
    # distance for every table row
    mocs = []
    areas = []
    es = []

    for row_index in range(len(table)):
        moc, area = compute_moc(table, row_index)
        es_r = compute_earth_shadow(table, row_index)

        mocs.append(moc)
        areas.append(area)
        es.append(es_r)

    # find matching plates by looking for footprint matches
    # within a given criteria.
    for i1 in range(len(table)):

#         # ignore rows already examined
#         if table['stc_polygon'][i1] == "":
#             continue

        # start from i1+1 to avoid self-reference
#         for i2 in range(i1+1, len(table)):
        for i2 in range(len(table)):
        
            # ignore if same row:
            if i1 == i2:
                continue

#             # ignore rows already selected
#             if table['stc_polygon'][i2] == "":
#                 continue
                
            # ignore if exposure times are different. They cause different
            # detection thresholds in sextractor.
            if table['exptime'][i2] != table['exptime'][i1]:
                continue

            moc_inter, area_inter = compute_moc_intersection(mocs[i1], mocs[i2])

            minarea = min(areas[i1], areas[i2])
            if area_inter > minarea * match_area:

                # in case of a match in area, see if this is a duplicate scan of 
                # the same plate; skip over duplicates
                if table['ut_mid'][i1] == table['ut_mid'][i2]:
                    continue
 
                t1 = Time(table['ut_mid'][i1])
                t2 = Time(table['ut_mid'][i2])
                dt = t2.mjd - t1.mjd
                
                fov1_1 = table['fov1'][i1] 
                fov1_2 = table['fov1'][i2] 
                
                processed = find_processed(table['plate_id'][i1], table['plate_id'][i2])

                result.add_row([
#                     processed,
#                     i1, i2,
                    table['plate_id'][i1], table['plate_id'][i2],
#                     table['exptime'][i1], table['exptime'][i2],
#                     table['emulsion'][i1], table['emulsion'][i2],
                    table['ut_mid'][i1], table['ut_mid'][i2],
                    fov1_1, fov1_2,
                    t1.mjd, dt, (area_inter / minarea),
                    es[i1]
                ])

                # mark row as already taken
                table['stc_polygon'][i2] = ""
    
    return result

In [7]:
def make_table():
#     table = Table(names=['processed', 'row1', 'row2', 'plate_id_1', 'plate_id_2', 'exptime_1', 'exptime_2',
#                           'emulsion_1', 'emulsion_2', 'ut_mid_1', 'ut_mid_2', 'delta_time', 'area'],
#                   dtype=['S1', 'i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'S1', 'S1', 'S1', 'S1', 'f4', 'f4'])
    
    table = Table(names=['plate_id_1', 'plate_id_2', 'ut_mid_1', 'ut_mid_2', 'fov1_1', 'fov1_2', 
                         'mjd_1', 'delta_time', 'area', 'es'],
                  dtype=['i4', 'i4', 'S1', 'S1', 'f4', 'f4', 'f4', 'f4', 'f4', 'f4'])

    table['fov1_1'].format = '%.2f' 
    table['fov1_2'].format = '%.2f' 
    table['mjd_1'].format = '%.2f' 
    table['delta_time'].format = '%.4f' 
    table['area'].format = '%.2f' 
    table['es'].format = '%.4f' 
    
    return table

In [8]:
def group_by_mjd(table, colname):
    
    groups = [0]
    current_group = 0
    mjds = table['mjd_1'].astype(int)
    
    for index in range(1, len(mjds)):
        
        if mjds[index] != mjds[index-1]:
            current_group += 1

        groups.append(current_group)
    
    table[colname] = groups
    
    return table    

In [9]:
def find_matching_plates(table, match_area=0.99):
    result = make_table()

    return scan_table(table, result, match_area)

In [10]:
def display_with_pandas(table):
    df = table.to_pandas()

    df['ut_mid_1'] = df['ut_mid_1'].apply(lambda x: x.decode('utf-8'))
    df['ut_mid_2'] = df['ut_mid_2'].apply(lambda x: x.decode('utf-8'))
    
    format_mapping = {
        'fov1_1': '{:.2f}'.format,  
        'fov1_2': '{:.2f}'.format,  
        'mjd_1': '{:.2f}'.format,  
        'delta_time': '{:.4f}'.format,  
        'area': '{:.2}'.format,
        'es': '{:.4}'.format,
    }
    styled_df = df.style.format(format_mapping)
    display(styled_df)

In [11]:
def find_processed(plate1, plate2):
    result = ""
    
    filename = fname(get_table_psf_nomatch(plate1, plate2))
    try:
        f = open(filename, 'r')
        result = 'YES'
        f.close()
    except FileNotFoundError:
        result = ''

    return result

## Read footprints table generated by APPLAUSE database

In [12]:
table_input = Table.read('footprints_5.csv', format='ascii.csv')

In [13]:
table_input[0:4]

telescope,ota_aperture,date_orig,ut_mid,exptime,plate_format,emulsion,plate_id,filename_scan,ra_icrs,dec_icrs,fov1,fov2,pixel_scale,naxis1,naxis2,stc_polygon,observatory,site_name,site_longitude,site_latitude
str23,float64,str10,str19,int64,str5,str45,int64,str68,float64,float64,float64,float64,float64,int64,int64,str92,str20,str26,float64,float64
Doppel-Reflektor,0.6,1934-01-20,1934-01-20 00:10:00,1200,09x12,--,61990,https://www.plate-archive.org/files/DR4/scans/HAM-DR/DRP0000c_x.fits,56.83299593063,24.11869187395,2.3150816,1.7282897,0.7296022,8608,11377,Polygon ICRS 58.07349 23.22626 55.55474 23.27329 55.57486 25.00187 58.12810 24.95400,Hamburger Sternwarte,"Hamburg-Bergedorf, Germany",10.242,53.482
Doppel-Reflektor,0.6,1934-01-20,1934-01-20 00:10:00,1200,09x12,--,61990,https://www.plate-archive.org/files/DR4/scans/HAM-DR/DRP0000c_x.fits,56.83719295039,24.11776814973,1.7439835,2.307311,0.72972935,8608,11377,Polygon ICRS 58.13113 24.95957 58.07095 23.21660 55.56056 23.26645 55.58575 25.01024,Hamburger Sternwarte,"Hamburg-Bergedorf, Germany",10.242,53.482
Doppel-Reflektor,0.6,1934-01-20,1934-01-20 00:10:00,1200,09x12,--,61990,https://www.plate-archive.org/files/DR4/scans/HAM-DR/DRP0000c_y.fits,56.83299593063,24.11869187395,2.3150816,1.7282897,0.7296022,11416,8533,Polygon ICRS 58.07349 23.22626 55.55474 23.27329 55.57486 25.00187 58.12810 24.95400,Hamburger Sternwarte,"Hamburg-Bergedorf, Germany",10.242,53.482
Doppel-Reflektor,0.6,1934-01-20,1934-01-20 00:10:00,1200,09x12,--,61990,https://www.plate-archive.org/files/DR4/scans/HAM-DR/DRP0000c_y.fits,56.83719295039,24.11776814973,1.7439835,2.307311,0.72972935,11416,8533,Polygon ICRS 58.13113 24.95957 58.07095 23.21660 55.56056 23.26645 55.58575 25.01024,Hamburger Sternwarte,"Hamburg-Bergedorf, Germany",10.242,53.482


In [14]:
len(table_input)

7314

## Group table rows with common footprint

Segregate by telescope and other relevant parameters.

Table column names are mostly the same names used in the APPLAUSE tables. Plus:

- **delta_time**: difference in MJD between the two plates
- **area**: overlapping area as a fraction of the area of the plate with smaller area
- **es**: distance from Earth's shadow (deg)
- **group**: used for grouping plates based on difference in MJD

In [15]:
# # 1m-Spiegelteleskop (this has a bad comma)
# table = table_input.copy()

# mask = table['telescope'] == "1m-Spiegelteleskop"
# table = table[mask]
# mask = table['plate_format'] == "13x18"
# table = table[mask]
# mask = table['exptime'] > 1000
# table = table[mask]

# table1 = find_matching_plates(table)

In [16]:
# table1

In [17]:
# Grosser Schmidt-Spiegel
table = table_input.copy()

mask = table['telescope'] == "Grosser Schmidt-Spiegel"
table = table[mask]
# mask = table['exptime'] >= 600
# table = table[mask]

table2 = find_matching_plates(table, match_area=0.5)

/Users/busko/Projects/software/miniconda3/envs/astro/lib/python3.10/site-packages/astropy/table/table.py:3223: UserWarning: Units from inserted quantities will be ignored.
  self.insert_row(len(self), vals, mask)


In [18]:
# build column that uniquely identifies plate combinations
new_id_strings = [f"{row['plate_id_1']}_{row['plate_id_2']}" for row in table2]
table2['plates'] = new_id_strings
table2 = unique(table2, keys=['plates'])

# only accept plates taken close in time (same night)
mask = table2['delta_time'] < 1.
table2 = table2[mask]
mask = table2['delta_time'] > -0.005
table2 = table2[mask]

# table2.sort(['plates', 'delta_time'])
table2.sort(['mjd_1', 'delta_time', 'ut_mid_2'])

table2 = group_by_mjd(table2, 'group')
t_grouped = table2.group_by('group')

print(len(t_grouped))
display_with_pandas(t_grouped)

126


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,8794,8795,1955-02-13 22:52:47,1955-02-13 23:29:56,2.15,2.87,35151.95,0.0258,1.0,19.29,8794_8795,0
1,8794,8796,1955-02-13 22:52:47,1955-02-14 00:00:36,2.15,5.78,35151.95,0.0471,1.0,19.29,8794_8796,0
2,8795,8796,1955-02-13 23:29:56,1955-02-14 00:00:36,2.87,5.78,35151.98,0.0213,1.0,18.63,8795_8796,0
3,8874,8875,1955-05-19 21:41:56,1955-05-19 22:11:51,5.78,5.78,35246.90,0.0208,0.99,51.38,8874_8875,1
4,9039,9040,1956-01-31 17:47:18,1956-01-31 18:30:41,5.78,5.79,35503.74,0.0301,0.99,87.35,9039_9040,2
5,9040,9012,1956-01-31 18:30:41,1956-02-01 17:52:07,5.79,5.78,35503.77,0.9732,0.99,86.41,9040_9012,2
6,9012,9013,1956-02-01 17:52:07,1956-02-01 18:35:14,5.78,5.77,35504.75,0.0299,1.0,87.83,9012_9013,3
7,9016,9017,1956-02-17 00:40:07,1956-02-17 01:06:48,5.78,5.77,35520.03,0.0185,0.99,35.23,9016_9017,4
8,9095,9096,1956-04-04 22:23:15,1956-04-04 22:50:11,5.68,5.78,35567.93,0.0187,1.0,38.44,9095_9096,5
9,9099,9100,1956-04-07 20:30:29,1956-04-07 21:34:33,5.78,5.78,35570.86,0.0445,1.0,68.14,9099_9100,6


Based on listing above, we pick up groups with 3 or more plates taken on the same night, or at most 1 night away.

Disregard duplications and redundancies in the groups table listing above. They are removed by the code when building the plate ID sequences. 

We can copy the sequences to the *sequences* entry in the. *settings.py* file, for more permanent use.

In [19]:
last_group = table2['group'][-1]
all_groups = []

for group in range(last_group+1):

    group_list = []

    mask = table2['group'] == group
    table2_a = table2[mask]
    
    if len(table2_a) >= 3:
        display_with_pandas(table2_a)
        
        all_plates = list(table2_a['plate_id_1']) + list(table2_a['plate_id_2'])
        group_list.extend(all_plates)
        group_list = list(dict.fromkeys(group_list)) # removing duplicates in group
        group_list.sort()
        
        all_groups.append(group_list)

print()
print("--------")
print("Interesting sequences of plates:")
print()
for seqno, group in enumerate(all_groups):
    print("'seq %02i': %s," % (seqno, str(group)))

,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,8794,8795,1955-02-13 22:52:47,1955-02-13 23:29:56,2.15,2.87,35151.95,0.0258,1.0,19.29,8794_8795,0
1,8794,8796,1955-02-13 22:52:47,1955-02-14 00:00:36,2.15,5.78,35151.95,0.0471,1.0,19.29,8794_8796,0
2,8795,8796,1955-02-13 23:29:56,1955-02-14 00:00:36,2.87,5.78,35151.98,0.0213,1.0,18.63,8795_8796,0


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9167,9168,1956-05-29 22:44:21,1956-05-29 23:12:56,5.77,5.78,35622.95,0.0198,0.52,76.22,9167_9168,7
1,9167,9169,1956-05-29 22:44:21,1956-05-29 23:30:43,5.77,5.77,35622.95,0.0322,0.52,76.22,9167_9169,7
2,9168,9169,1956-05-29 23:12:56,1956-05-29 23:30:43,5.78,5.77,35622.97,0.0123,1.0,74.21,9168_9169,7


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9245,9246,1956-09-25 19:00:47,1956-09-25 19:43:40,5.69,5.77,35741.79,0.0298,1.0,69.35,9245_9246,12
1,9245,9247,1956-09-25 19:00:47,1956-09-25 20:29:33,5.69,5.69,35741.79,0.0616,1.0,69.35,9245_9247,12
2,9246,9247,1956-09-25 19:43:40,1956-09-25 20:29:33,5.77,5.69,35741.82,0.0319,1.0,68.44,9246_9247,12


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9313,9315,1956-12-03 17:15:31,1956-12-03 18:06:23,2.16,2.16,35810.72,0.0353,1.0,62.87,9313_9315,15
1,9313,9317,1956-12-03 17:15:31,1956-12-03 18:56:48,2.16,2.16,35810.72,0.0703,1.0,62.87,9313_9317,15
2,9313,9318,1956-12-03 17:15:31,1956-12-03 19:26:11,2.16,2.16,35810.72,0.0907,1.0,62.87,9313_9318,15
3,9313,9319,1956-12-03 17:15:31,1956-12-03 20:27:18,2.16,2.14,35810.72,0.1332,1.0,62.87,9313_9319,15
4,9313,9320,1956-12-03 17:15:31,1956-12-03 20:55:56,2.16,2.14,35810.72,0.1531,1.0,62.87,9313_9320,15
5,9315,9317,1956-12-03 18:06:23,1956-12-03 18:56:48,2.16,2.16,35810.75,0.0350,1.0,61.75,9315_9317,15
6,9315,9318,1956-12-03 18:06:23,1956-12-03 19:26:11,2.16,2.16,35810.75,0.0554,1.0,61.75,9315_9318,15
7,9315,9319,1956-12-03 18:06:23,1956-12-03 20:27:18,2.16,2.14,35810.75,0.0979,1.0,61.75,9315_9319,15
8,9315,9320,1956-12-03 18:06:23,1956-12-03 20:55:56,2.16,2.14,35810.75,0.1177,1.0,61.75,9315_9320,15
9,9317,9318,1956-12-03 18:56:48,1956-12-03 19:26:11,2.16,2.16,35810.79,0.0204,1.0,60.63,9317_9318,15


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9322,9323,1956-12-06 18:54:56,1956-12-06 19:19:52,2.88,2.13,35813.79,0.0173,0.99,61.95,9322_9323,16
1,9322,9324,1956-12-06 18:54:56,1956-12-06 19:44:24,2.88,2.87,35813.79,0.0344,1.0,61.95,9322_9324,16
2,9322,9325,1956-12-06 18:54:56,1956-12-06 20:08:14,2.88,2.87,35813.79,0.0509,1.0,61.95,9322_9325,16
3,9322,9327,1956-12-06 18:54:56,1956-12-06 22:14:24,2.88,2.77,35813.79,0.1385,1.0,61.95,9322_9327,16
4,9322,9321,1956-12-06 18:54:56,1956-12-07 18:16:06,2.88,2.88,35813.79,0.9730,1.0,61.95,9322_9321,16
5,9323,9324,1956-12-06 19:19:52,1956-12-06 19:44:24,2.13,2.87,35813.80,0.0170,0.99,61.4,9323_9324,16
6,9323,9325,1956-12-06 19:19:52,1956-12-06 20:08:14,2.13,2.87,35813.80,0.0336,1.0,61.4,9323_9325,16
7,9323,9327,1956-12-06 19:19:52,1956-12-06 22:14:24,2.13,2.77,35813.80,0.1212,1.0,61.4,9323_9327,16
8,9323,9321,1956-12-06 19:19:52,1956-12-07 18:16:06,2.13,2.88,35813.80,0.9557,1.0,61.4,9323_9321,16
9,9324,9325,1956-12-06 19:44:24,1956-12-06 20:08:14,2.87,2.87,35813.82,0.0166,1.0,60.91,9324_9325,16


...

,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9534,9535,1957-05-24 23:58:49,1957-05-25 00:19:45,5.77,5.78,35983.00,0.0145,1.0,73.02,9534_9535,31
1,9536,9537,1957-05-25 22:15:39,1957-05-25 22:33:36,5.75,5.75,35983.93,0.0125,1.0,118.2,9536_9537,31
2,9537,9543,1957-05-25 22:33:36,1957-05-26 22:22:42,5.75,5.73,35983.94,0.9924,0.87,118.2,9537_9543,31
3,9538,9539,1957-05-25 23:05:01,1957-05-25 23:21:58,5.77,5.78,35983.96,0.0118,1.0,69.4,9538_9539,31
4,9538,9540,1957-05-25 23:05:01,1957-05-25 23:39:25,5.77,5.77,35983.96,0.0239,1.0,69.4,9538_9540,31
5,9539,9540,1957-05-25 23:21:58,1957-05-25 23:39:25,5.78,5.77,35983.97,0.0121,1.0,69.47,9539_9540,31


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9542,9548,1957-05-26 00:22:48,1957-05-26 23:58:56,5.78,5.70,35984.02,0.9834,1.0,74.99,9542_9548,32
1,9542,9549,1957-05-26 00:22:48,1957-05-27 00:15:53,5.78,5.70,35984.02,0.9952,1.0,74.99,9542_9549,32
2,9543,9544,1957-05-26 22:22:42,1957-05-26 22:41:39,5.73,5.74,35984.93,0.0132,1.0,118.2,9543_9544,32
3,9545,9546,1957-05-26 23:06:04,1957-05-26 23:26:01,5.77,5.78,35984.96,0.0139,1.0,71.68,9545_9546,32
4,9545,9547,1957-05-26 23:06:04,1957-05-26 23:41:59,5.77,5.77,35984.96,0.0249,1.0,71.68,9545_9547,32
5,9546,9547,1957-05-26 23:26:01,1957-05-26 23:41:59,5.78,5.77,35984.98,0.0111,1.0,71.78,9546_9547,32


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9550,9551,1957-05-30 23:11:10,1957-05-30 23:30:44,5.76,5.78,35988.96,0.0136,1.0,69.61,9550_9551,34
1,9550,9552,1957-05-30 23:11:10,1957-05-30 23:47:46,5.76,5.72,35988.96,0.0254,1.0,69.61,9550_9552,34
2,9551,9552,1957-05-30 23:30:44,1957-05-30 23:47:46,5.78,5.72,35988.98,0.0118,1.0,69.7,9551_9552,34


,plate_id_1,plate_id_2,ut_mid_1,ut_mid_2,fov1_1,fov1_2,mjd_1,delta_time,area,es,plates,group
0,9553,9555,1957-05-31 00:07:53,1957-05-31 23:06:21,5.77,5.78,35989.00,0.9573,0.59,75.42,9553_9555,35
1,9553,9556,1957-05-31 00:07:53,1957-05-31 23:24:19,5.77,5.77,35989.00,0.9697,0.6,75.42,9553_9556,35
2,9555,9556,1957-05-31 23:06:21,1957-05-31 23:24:19,5.78,5.77,35989.96,0.0125,0.99,75.28,9555_9556,35
3,9557,9558,1957-05-31 23:41:16,1957-05-31 23:59:13,5.77,5.78,35989.99,0.0125,1.0,71.9,9557_9558,35



--------
Interesting sequences of plates:

'seq 00': [8794, 8795, 8796],
'seq 01': [9167, 9168, 9169],
'seq 02': [9245, 9246, 9247],
'seq 03': [9313, 9315, 9316, 9317, 9318, 9319, 9320],
'seq 04': [9321, 9322, 9323, 9324, 9325, 9326, 9327, 9328],
'seq 05': [9344, 9345, 9346, 9347, 9348, 9349, 9350],
'seq 06': [9341, 9342, 9343],
'seq 07': [9473, 9474, 9475],
'seq 08': [9488, 9489, 9490, 9491, 9492],
'seq 09': [9521, 9522, 9523],
'seq 10': [9531, 9533, 9534, 9535, 9536],
'seq 11': [9534, 9535, 9536, 9537, 9538, 9539, 9540, 9543],
'seq 12': [9542, 9543, 9544, 9545, 9546, 9547, 9548, 9549],
'seq 13': [9550, 9551, 9552],
'seq 14': [9553, 9555, 9556, 9557, 9558],
